1. Number of updated projects of both sheets since and before (by donor agency and by SWG)(The number of updated projects refers to both the online Excel sheet and Google form)
2. Number of updated projects on the online Excel sheet only (whose online form is not updated)( by donor agency and by SWG)
3. Number of new projects and list registered ( by donor agency and by SWG)
4. Number of not updated projects ( by donor agency and by SWG)
5. Can we get an Excel sheet under each SWG updated and not updated project info of both online and Excel sheet, updated fields colored?
6. Can we get all SWG info in one file in the above similar request?

On the AMP system or new system or online form: create a links for the user like existing since, existing before, new project, select donor org (select yours in the case of multi), etc. when all infor filled filter out specific project and take the data encoder to relevant page, meaning page with filtered out projects.

In [1]:
import pandas as pd
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
# improt tqdm for progress bar
from tqdm import tqdm

# Define your functions get_ids_and_titles and compare_stat here
# get_ids_and_titles
def get_ids_and_titles(original, latest):
    # Retrieve values from specific columns
    original_titles = original.iloc[7:, 2].values
    latest_titles = latest.iloc[7:, 2].values
    original_ids = original.iloc[7:, 7].values
    latest_ids = latest.iloc[7:, 7].values

    return original_titles, latest_titles, original_ids, latest_ids   

# compare_stat
def compare_stat(original, latest):
    # 3. Create dictionaries for categorizing projects
    new_projs = {}
    original_not_updated_projs = {}
    original_updated_projs = {}
    deleted_projs = {}

    _, latest_titles, original_ids, latest_ids = get_ids_and_titles(original, latest)

    # get all ids which are in original but not in latest
    for i, original_id in enumerate(original_ids):
        if original_id not in latest_ids:
            deleted_projs[original_id] = original.iloc[i + 7]

    # 4. Iterate over latest IDs for comparison
    for i, latest_id in enumerate(latest_ids): 
        # declare empty index and row variables   

        if latest_id in original_ids:
            # Find the index of the row with the matching ID in original DataFrame
            original_index = original[original.iloc[:, 7] == latest_id].index[0]
            original_row = original.iloc[original_index]

            # Extract and compare the entire rows
            latest_index = latest[latest.iloc[:, 7] == latest_id].index[0]
            latest_row = latest.iloc[latest_index]

            # Categorize based on comparison
            if original_row.equals(latest_row):
                original_not_updated_projs[latest_id] = original_row
            else:
                original_updated_projs[latest_id] = latest_row

        # enter here if id null or space or special characters in the ids excluding hyphen '-', id not in 87... or new-... or empty
        elif pd.isna(latest_id) or latest_id.isspace() or bool(re.search(r'[^a-zA-Z0-9\-]', latest_id)) or latest_id is '':
            if latest_titles[i] not in new_projs: # latest_titles[i] not in new_project
                # new_project as id: corresponding row including values of all columns
                new_projs[latest_titles[i]] = latest.iloc[i + 7]
            new_projs[latest_titles[i]] = latest.iloc[i + 7]
        else:
            # print id and say id error
            print(f'###### ID ERROR. Id: {latest_id} Title: {latest_titles[i]} ######')                       

    return new_projs, deleted_projs, original_not_updated_projs, original_updated_projs

def process_files(orig_file, lat_file, output_fname):
    original_sinc = pd.read_excel(orig_file, sheet_name=sincesht)
    latest_sinc = pd.read_excel(lat_file, sheet_name=sincesht)

    original_befo = pd.read_excel(orig_file, sheet_name=beforesht)
    latest_befo = pd.read_excel(lat_file, sheet_name=beforesht)

    original_new = pd.read_excel(orig_file, sheet_name=newsht)
    latest_new = pd.read_excel(lat_file, sheet_name=newsht)

    # get comparison statistics
    new_project_sinc, deleted_project_sinc, original_not_updated_projs_sinc, original_updated_projs_sinc = compare_stat(original_sinc, latest_sinc)
    new_project_befo, deleted_project_befo, original_not_updated_projs_befo, original_updated_projs_befo = compare_stat(original_befo, latest_befo)
    new_project_new, deleted_project_new, original_not_updated_projs_new, original_updated_projs_new = compare_stat(original_new, latest_new)

    
    # write to excel
    with pd.ExcelWriter(output_fname) as writer:
        pd.DataFrame.from_dict(new_project_sinc, orient='index').to_excel(writer, sheet_name='New Projs Since 2017')
        pd.DataFrame.from_dict(deleted_project_sinc, orient='index').to_excel(writer, sheet_name='Deleted Projs Since 2017')
        pd.DataFrame.from_dict(original_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Updated Projs Since 2017')
        pd.DataFrame.from_dict(original_not_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Not Updated Projs Since 2017')

        pd.DataFrame.from_dict(new_project_befo, orient='index').to_excel(writer, sheet_name='New Projs Before 2017')
        pd.DataFrame.from_dict(deleted_project_befo, orient='index').to_excel(writer, sheet_name='Deleted Projs Before 2017')
        pd.DataFrame.from_dict(original_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Updated Projs Before 2017')
        pd.DataFrame.from_dict(original_not_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Not Updated Projs Before 2017')

        pd.DataFrame.from_dict(new_project_new, orient='index').to_excel(writer, sheet_name='New Projs new')
        # pd.DataFrame.from_dict(deleted_project_new, orient='index').to_excel(writer, sheet_name='Deleted Projs new')
        # pd.DataFrame.from_dict(original_updated_projs_new, orient='index').to_excel(writer, sheet_name='Updated Projs new')
        # pd.DataFrame.from_dict(original_not_updated_projs_new, orient='index').to_excel(writer, sheet_name='Not Updated Projs new')
        
    # read excel file back and change the column names for all sheets
    df = pd.read_excel(output_fname, sheet_name=None)

    sheets_to_exclude = []
    for sheet in df:
        # Check if the sheet has zero columns before slicing
        if df[sheet].shape[1] == 0:
            sheets_to_exclude.append(sheet)
            continue

        # Now safe to remove the first two and the last one columns
        df[sheet] = df[sheet].iloc[:, 2:-1]

        # Further checks and adjustments
        if df[sheet].shape[1] > len(cols):
            # Slice to only the first 19 columns if there are more
            df[sheet] = df[sheet].iloc[:, :len(cols)]
        elif df[sheet].shape[1] < len(cols):
            # Add missing columns if the DataFrame has fewer than 19 columns
            for col in cols[df[sheet].shape[1]:]:
                df[sheet][col] = pd.NA

        df[sheet].columns = cols

    # Concatenate specific sheets into one DataFrame
    sheets_to_merge = ['New Projs Since 2017', 'New Projs Before 2017', 'New Projs new']
    merged_df = pd.concat([df[s] for s in sheets_to_merge if s not in sheets_to_exclude], axis=0)
    df['New Projects'] = merged_df[cols]

    # Exclude original sheets that were merged
    sheets_to_exclude.extend(sheets_to_merge)

    # write to excel
    with pd.ExcelWriter(output_fname) as writer:
        for sheet in df:
            if sheet in sheets_to_exclude:
                print(f'###### Empty Sheet {sheet} has been excluded ######')
                continue
            df[sheet].to_excel(writer, sheet_name=sheet, index=False)
            # Access the workbook and the sheet
            
            worksheet = writer.sheets[sheet]

            # highlight the header cells
            # Define header format
            header_fill = PatternFill(start_color="D7E4BC", end_color="D7E4BC", fill_type="solid")
            thin_border = Border(left=Side(style='thin'), 
                                right=Side(style='thin'), 
                                top=Side(style='thin'), 
                                bottom=Side(style='thin'))
            header_font = Font(bold=True)
            header_alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)

            # Apply header format
            for cell in worksheet[1]:
                cell.fill = header_fill
                cell.border = thin_border
                cell.font = header_font
                cell.alignment = header_alignment

            # add filter to the header
            worksheet.auto_filter.ref = "A1:S1"

            # freeze the first row
            worksheet.freeze_panes = 'A2' 

            # Set column widths
            high = ['A', 'B']  
            medium = ['F', 'G', 'H', 'L', 'M', 'K',  'I', 'J', 'Q', 'R', 'S', 'T'] 
            low = ['C', 'D', 'E', 'N', 'O', 'P']

            for col in high:
                worksheet.column_dimensions[col].width = 30

            for col in medium:
                worksheet.column_dimensions[col].width = 18 

            for col in low:
                worksheet.column_dimensions[col].width = 12    

# Define column names and sheet names
cols = ['SWG', 'Project Title', 'Status', 'On/Off/Treasury Budget', 'Humanitarian Aid', 'Project description', 
        'Project ID (in AMP)', 'Actual Approval Date', 'Actual Start Date', 'Actual Closure Date', 
        'Donor Group', 'Donor Agency', 'Implementing Agency', 'Type of Assistance', 'Mode of Payment', 
        'Aid Modality', 'Actual Commitments', 'Actual Disbursements', 'Undisbursed Balance']

sincesht = 'Existing projects approved sinc'
beforesht = 'Existing projects approved befo'
newsht = 'Record new projects'

# List of file pairs and their corresponding output files
file_pairs = [
    ('ODA_Original_and_Latest/OrigHPN.xlsx', 'ODA_Original_and_Latest/LatHPN.xlsx', 'Report/HPN.xlsx'),
    ('ODA_Original_and_Latest/OrigBSD.xlsx', 'ODA_Original_and_Latest/LatBSD.xlsx', 'Report/BSD.xlsx'),
    ('ODA_Original_and_Latest/OrigREDFS.xlsx', 'ODA_Original_and_Latest/LatREDFS.xlsx', 'Report/REDFS.xlsx'),
    # Add more file pairs as needed
]

tqdm.write('Processing files...')
for orig_file, lat_file, output_fname in tqdm(file_pairs):
    process_files(orig_file, lat_file, output_fname)
tqdm.write('Done!')    


<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\wesag\AppData\Local\Temp\ipykernel_26812\3595527233.py:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pd.isna(latest_id) or latest_id.isspace() or bool(re.search(r'[^a-zA-Z0-9\-]', latest_id)) or latest_id is '':


Processing files...


 33%|███▎      | 1/3 [00:02<00:05,  2.52s/it]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######


 67%|██████▋   | 2/3 [00:04<00:02,  2.01s/it]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet Deleted Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet Updated Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######


100%|██████████| 3/3 [00:06<00:00,  2.29s/it]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet Deleted Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet Updated Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######
Done!


In [2]:
# Merge all Excel files in a directory into one
import pandas as pd
import os
from glob import glob

# Directory containing Excel files
directory = 'Report'

# Find all Excel files in the directory
excel_files = glob(os.path.join(directory, '*.xlsx'))

# Dictionary to store data from each sheet
merged_data = {}

# Process each file
for file in excel_files:
    # Read each sheet in the file
    xls = pd.ExcelFile(file)
    for sheet_name in xls.sheet_names:
        # Read the sheet
        df = pd.read_excel(file, sheet_name=sheet_name)
        
        # If the sheet name is already in merged_data, append the new data
        if sheet_name in merged_data:
            merged_data[sheet_name] = pd.concat([merged_data[sheet_name], df])
        else:
            merged_data[sheet_name] = df

# Write merged data to a new Excel file
with pd.ExcelWriter('Merged.xlsx') as writer:
    for sheet_name, data in merged_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

In [3]:
import pandas as pd

# Paths of the Excel files
file1_path = 'Merged.xlsx'
file2_path = 'AMEResponse.xlsx'
output_file_path = 'Response Mereged.xlsx'

file2_df = pd.read_excel(file2_path, sheet_name='Form Responses 1')

# Read all sheets from the first file
file1_sheets = pd.read_excel(file1_path, sheet_name=None)



In [4]:
# get column names of the first sheet with corresponding indexes
col_names = {}
for col in file1_sheets['New Projects'].columns:
    col_names[col] = file1_sheets['New Projects'].columns.get_loc(col)

# get column names of the second file with corresponding indexes
col_names2 = {}
for col in file2_df.columns:
    col_names2[col] = file2_df.columns.get_loc(col)

In [9]:
# print the first 5 values of column names and indexes
for key, value in list(col_names.items())[:5]:
    print(key, value)

print('-------------------')
# print the first 5 values of column names and indexes
for key, value in list(col_names2.items())[:5]:
    print(key, value)

SWG 0
Project Title 1
Status 2
On/Off/Treasury Budget 3
Humanitarian Aid 4
-------------------
Timestamp 0
Email Address 1
AMP- Project ID - Please do not modify (this ID is for internal use only) 2
Project Title 3
Project ID (to be entered as in Project Document) 4


In [8]:

# Create a writer object for the output file
with pd.ExcelWriter(output_file_path) as writer:
    # Loop through each sheet in file1
    for sheet_name, file1_df in file1_sheets.items():
        # Find matching rows
        matching_rows = file1_df[file1_df[file1_df.columns[1]].isin(file2_df[file2_df.columns[3]])]
        print(f'Found {matching_rows.shape[0]} matching rows in {sheet_name}')

        # or get the values of the first column of the rows and compare with the values of the third column of the second file
        
        # Write to the new Excel file
        matching_rows.to_excel(writer, sheet_name=sheet_name, index=False)


Found 0 matching rows in Updated Projs Since 2017
Found 0 matching rows in Not Updated Projs Since 2017
Found 0 matching rows in Not Updated Projs Before 2017
Found 9 matching rows in New Projects
Found 0 matching rows in Deleted Projs Since 2017
Found 0 matching rows in Updated Projs Before 2017


In [7]:
# Convert the third column to string and filter out empty strings
file2_df[file2_df.columns[3]] = file2_df[file2_df.columns[3]].astype(str)
file2_df = file2_df[file2_df[file2_df.columns[3]].str.strip() != '']

# Read all sheets from the first file
file1_sheets = pd.read_excel(file1_path, sheet_name=None)
similar_str = []
# Create a writer object for the output file
with pd.ExcelWriter(output_file_path) as writer:
    # Loop through each sheet in file1
    for sheet_name, file1_df in file1_sheets.items():
        # Convert the first column to string and filter out empty strings
        file1_df[file1_df.columns[1]] = file1_df[file1_df.columns[1]].astype(str)
        file1_df = file1_df[file1_df[file1_df.columns[1]].str.strip() != '']


        str1 = file2_df[file2_df.columns[3]].values.astype(str)
        str2 = file1_df[file1_df.columns[1]].values.astype(str)

        for i in range(len(str1)):
            str1[i] = str1[i].strip()
        for i in range(len(str2)):
            str2[i] = str2[i].strip()

        for i in range(len(str1)):
            for j in range(len(str2)):
                if str1[i] == str2[j] or str1[i] in str2[j] or str2[j] in str1[i]:
                    # print(str1[i] \t str2[j])
                    # print(f'\n Before {str1[i]} <==> {str2[j]} ==> {sheet_name}')

                    # trim the string to the first character until the first space '...' is found
                    str1[i] = str1[i][:str1[i].find('...')]
                    # print(f'\n After {str1[i]} <==> {str2[j]} ==> {sheet_name}')

                    similar_str.append(str1[i])                   
                            
        matching_rows = file1_df[file1_df[file1_df.columns[1]].isin(similar_str)]
        print(f'Found {matching_rows.shape[0]} matching rows in {sheet_name}')

        # Find rows where the first column of file1 matches the third column of file2
        # matching_rows = file1_df[file1_df[file1_df.columns[1]].isin(file2_df[file2_df.columns[3]])]

        # print title values of the matching rows
        # print(matching_rows[file1_df.columns[1]].values)



        # Write matching rows to the new Excel file
        matching_rows.to_excel(writer, sheet_name=sheet_name, index=False)

print(len(similar_str[:6]))

Found 4 matching rows in Updated Projs Since 2017
Found 1 matching rows in Not Updated Projs Since 2017
Found 0 matching rows in Not Updated Projs Before 2017
Found 3 matching rows in New Projects
Found 0 matching rows in Deleted Projs Since 2017
Found 0 matching rows in Updated Projs Before 2017
6
